In [0]:
%sql
DESCRIBE HISTORY table_name;


In [0]:
from delta.tables import DeltaTable
history = DeltaTable.forName(spark, "table_name").history()
display(history)


In [0]:
%sql
SELECT * FROM table_name VERSION AS OF 5;


In [0]:
df_old = spark.read.format("delta") \
  .option("timestampAsOf", "2026-01-01") \
  .table("table_name")


Data Correction Strategies
Depending on the scale of the error, use one of these three methods:
A. Full Rollback (Permanent)
Use RESTORE to permanently revert the entire table to a previous state. 
SQL: RESTORE TABLE table_name TO VERSION AS OF 10;
Python: deltaTable.restoreToVersion(10)
B. Partial Correction (Specific Rows)
If only certain records are incorrect, use MERGE to pull the correct values from the historical version back into the current table.

In [0]:
MERGE INTO table_name target
USING (SELECT * FROM table_name VERSION AS OF 5) source
ON target.id = source.id
WHEN MATCHED AND target.error_column != source.error_column THEN
  UPDATE SET target.error_column = source.error_column;



C. Fixing Accidental Deletes
Recover deleted rows by inserting them back from a previous version.

In [0]:
%sql
INSERT INTO table_name
SELECT * FROM table_name VERSION AS OF 5
WHERE id NOT IN (SELECT id FROM table_name);


Important Constraints for 2026

Retention Period: You can only travel back as far as your VACUUM settings allow (default is 7 days).

Change Data Feed (CDF): For precise auditing of row-level changes between versions, ensure delta.enableChangeDataFeed = true is enabled on the table.